# 리콜 내부 정보

In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import pandas as pd
import chromedriver_autoinstaller

# 크롬 드라이버 자동 설치 및 설정
chromedriver_autoinstaller.install()
driver = webdriver.Chrome()

# 첫 페이지 URL 설정
url = "https://www.car.go.kr/ri/stat/list.do"

# 첫 페이지 로드
driver.get(url)
time.sleep(30)  # 페이지 로딩을 기다림

def get_detail_data(driver):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    subject = soup.select_one('div.subject').get_text(strip=True) if soup.select_one('div.subject') else ''
    rows = soup.select('tbody.uk-text-left tr')
    
    detail_data = {
        '제작(수입)사': '',
        '차명': '',
        '생산기간': '',
        '시정기간': '',
        '대상수량': '',
        '장치분류': '',
        '결함내용': '',
        '시정방법': '',
        '기타문의': ''
    }
    
    for row in rows:
        ths = row.select('th')
        tds = row.select('td')
        for th, td in zip(ths, tds):
            th_text = th.get_text(strip=True)
            td_text = td.get_text(strip=True)
            detail_data[th_text] = td_text
    
    return subject, detail_data

def get_page_data(driver):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    items = soup.select('ul.board-hrznt-list > li')
    data = []

    for item in items:
        title = item.select_one('strong').get_text(strip=True) if item.select_one('strong') else ''
        ministry = item.select_one('ol > li').get_text(strip=True) if item.select_one('ol > li') else ''
        date = item.select('ol > li')[1].get_text(strip=True) if len(item.select('ol > li')) > 1 else ''
        onclick_attr = item.select_one('a.new')['onclick'] if item.select_one('a.new') else item.select_one('a')['onclick']
        
        if onclick_attr:
            try:
                detail_view_js = onclick_attr.split("'")[1]
                driver.execute_script(f"$main.event.detailView('{detail_view_js}','O');")
                time.sleep(3)  # 페이지 로딩을 기다림

                subject, detail_data = get_detail_data(driver)
                detail_data['Title'] = title
                detail_data['Ministry'] = ministry
                detail_data['Date'] = date
                detail_data['Subject'] = subject
                data.append(detail_data)
                
                driver.back()
                time.sleep(3)  # 페이지 로딩을 기다림
            except Exception as e:
                print(f"상세 페이지로 이동 실패: {e}")
    
    return data

all_data = []

# 40페이지까지 크롤링 (예: 10단위 페이지를 고려)
MAX_PAGES = 763
for page in range(1, MAX_PAGES + 1):  # 페이지 범위를 줄였습니다. 실제 사용 시 원하는 범위로 변경하세요.
    page_data = get_page_data(driver)
    all_data.extend(page_data)
    
    if page < 763:
        try:
            # 현재 페이지가 10의 배수일 경우 "다음페이지" 클릭
            if page % 10 == 0:
                next_page = driver.find_element(By.XPATH, "//a[@title='다음페이지']")
            else:
                next_page = driver.find_element(By.XPATH, f"//a[contains(text(),'{page + 1}')]")
            
            driver.execute_script("arguments[0].click();", next_page)
            time.sleep(3)  # 페이지 로딩을 기다림
        except Exception as e:
            print(f"페이지 {page + 1}로 이동 실패: {e}")
            break

# 크롤링 완료 후 브라우저 종료
driver.quit()

# 데이터를 DataFrame으로 변환
if all_data:
    df = pd.DataFrame(all_data)
else:
    df = pd.DataFrame(columns=['Title', 'Ministry', 'Date', 'Subject', '제작(수입)사', '차명', '생산기간', '시정기간', '대상수량', '장치분류', '결함내용', '시정방법', '기타문의'])

# DataFrame 출력
print(df)

# 데이터 저장
df.to_csv('car_recall_data.csv', index=False)
print("데이터가 'car_recall_data.csv' 파일에 저장되었습니다.")


            제작(수입)사                                            차명  \
0     메르세데스벤츠코리아(주)             A 220 Hatch , A 250 , A 220 Sedan   
1     메르세데스벤츠코리아(주)                                  C 300 4MATIC   
2       비엠더블유코리아(주)  BMW X5 xDrive35d, BMW M550d xDrive, BMW 640i   
3       비엠더블유코리아(주)                     BMW 1 Series Sport 등 71차종   
4     메르세데스벤츠코리아(주)                           E 450 4MATIC 등 21차종   
...             ...                                           ...   
3805  메르세데스벤츠코리아(주)        S280, S320, S420, S500, S600, S600(쿠페)   
3806   스텔란티스코리아주식회사                                         스트라투스   
3807   스텔란티스코리아주식회사                                         닷지캐라반   
3808    (주)볼보자동차코리아                                      960, 850   
3809   스텔란티스코리아주식회사                                           체로키   

                         생산기간                     시정기간       대상수량  \
0     2018-06-21 ~ 2020-03-10             2024-08-09 ~    1,069 대   
1     2023-05-12 ~ 2023-12-02    